In [103]:
import pandas as pd
import pandasql as psql
from pandasql import sqldf
import numpy as np
import glob
import re

In [189]:
nation_code = pd.read_csv('staging_data/fifa_country_codes.csv')
nation_data = pd.read_csv('staging_data/combined_nation_data_cleaned.csv')

query = """
        SELECT DISTINCT country, code as nation_code
        FROM nation_code 
        order by country asc
        """
df_nation_1 = sqldf(query, locals())
df_nation_1['nation_id'] = range(1, len(df_nation_1) +1)
df_nation_1.to_csv(f'test.csv', index=False)
query = """
        SELECT b.nation_id, a.*, b.nation_code
        FROM nation_data a
        LEFT JOIN df_nation_1 b
                on a.nation=b.country
        order by nation, season asc
        """

result_df = sqldf(query, locals())

# Re-order the columns
result_df = result_df[['nation_id','nation','nation_code','confederation','nation_rank','nation_prev_rank','points','season']]

result_df.to_csv(f'curated_data/nation_data_dim.csv', index=False)

In [225]:
player_data = pd.read_csv('staging_data/combined_player_stats_cleaned.csv')


# Sorts the players alphabetically
query = """
        SELECT DISTINCT player, league
        FROM player_data 
        ORDER BY player asc
        """

result_df = sqldf(query, locals())

result_df['player_id'] = range(1, len(result_df) +1)

query = """

        SELECT  a.player_id, 
                b.player,
                b.nation,
                b.position,
                b.squad as club,
                b.age,
                b.year_of_birth,
                b.league,
                b.season
        FROM result_df a
        LEFT JOIN player_data b
            on a.player=b.player and a.league = b.league

"""
result_df2 = sqldf(query, locals())

result_df2['age'] = pd.to_numeric(result_df2['age'], errors='coerce').astype('Int64')
result_df2['year_of_birth'] = pd.to_numeric(result_df2['year_of_birth'], errors='coerce').astype('Int64')

# Building the Player Dimension Table
result_df2.to_csv(f'curated_data/player_data_dim.csv', index=False)
# result_df2.to_csv(f'test/player_data_dim.csv', index=False)


In [237]:
csvs  = glob.glob('staging_data/*combined_player*.csv')

for csv in csvs:

        match = re.search(r'player_(.*?)_cleaned', csv)
        keyword = match.group(1)

        player_data = pd.read_csv('staging_data/combined_player_stats_cleaned.csv')
        player_id = pd.read_csv('curated_data/player_data_dim.csv')

        # Building the Player Sub Dimension Tables as part of the snowflake schema

        # Sub Dim
        player_sub_data = pd.read_csv(f'staging_data/combined_player_{keyword}_cleaned.csv')

        if 'max_value' in csv:

                query = """
        
                SELECT  a.player_id,
                        a.player as player_1, 
                        a.nation as nation_1,
                        a.position as position_1,
                        a.club as club_1,
                        a.age as age_1,
                        a.year_of_birth as year_of_birth_1,
                        b.max_market_value,
                        a.league as league_1,
                        a.season as season_1
                FROM player_id a
                LEFT JOIN player_sub_data b
                on a.player=b.player and a.season=b.season

                """
                df = sqldf(query, locals())

                df = df.rename(columns = {'player_1':'player','nation_1': 'nation','position_1': 'position','club_1': 'club',
                                        'age_1': 'age','year_of_birth_1': 'year_of_birth','league_1': 'league', 'season_1': 'season' })

                # Replace missing values with -1
                df['max_market_value'] = df['max_market_value'].fillna(-1)
                df['max_market_value'] = pd.to_numeric(df['max_market_value'], errors='coerce').astype('Int64')
        else:

                query = """
                
                        SELECT  a.player_id,
                                a.player as player_1, 
                                a.nation as nation_1,
                                a.position as position_1,
                                a.club as club_1,
                                a.age as age_1,
                                a.year_of_birth as year_of_birth_1,
                                a.league as league_1,
                                a.season as season_1,
                                b.*
                        FROM player_id a
                        LEFT JOIN player_sub_data b
                        on a.player=b.player and a.league=b.league and a.season=b.season

                        """

                df = sqldf(query, locals())

                df = df.drop(columns = ['player','nation','position','squad','age','year_of_birth','league','season'])

                df = df.rename(columns = {'player_1':'player','nation_1': 'nation','position_1': 'position','club_1': 'club',
                                        'age_1': 'age','year_of_birth_1': 'year_of_birth','league_1': 'league', 'season_1': 'season' })

        df['age'] = pd.to_numeric(df['age'], errors='coerce').astype('Int64')
        df['year_of_birth'] = pd.to_numeric(df['year_of_birth'], errors='coerce').astype('Int64')

        if keyword == 'defense':
                keyword = 'defence'
        else:
                pass

        if keyword == 'stats':
                df.to_csv(f'curated_data/player_overall_{keyword}_data_subdim.csv', index=False)
        else:
                df.to_csv(f'curated_data/player_{keyword}_data_subdim.csv', index=False)

        print(f'{keyword} table written to output location')


max_value table written to output location


stats table written to output location
keeper table written to output location
misc table written to output location
shooting table written to output location
passing table written to output location
defence table written to output location


In [137]:
team_data = pd.read_csv('staging_data/combined_team_stats_cleaned.csv')

query = """
        SELECT DISTINCT squad as club
        FROM team_data
        ORDER BY squad
        """

team_df1 = sqldf(query, locals())

team_df1['club_id'] = range(1, len(team_df1) +1)

# Sorts the players alphabetically
query = """
        SELECT  b.club_id,
                a.squad as club,
                a.`#_of_players` as num_of_players,
                a.average_age as avg_age,
                a.possession as avg_possession,
                a.matches_played,
                a.league,
                a.season
        FROM    team_data a
        LEFT JOIN team_df1 b
            on a.squad=b.club
        ORDER BY squad
        """

team_df2 = sqldf(query, locals())


team_df2['avg_possession'] = pd.to_numeric(team_df2['avg_possession'], errors='coerce').astype('Int64')

# Building the Club Dimension Table
team_df2.to_csv(f'curated_data/club_data_dim.csv', index=False)
# result_df2.to_csv(f'test/player_data_dim.csv', index=False)


In [222]:
csvs  = glob.glob('staging_data/*combined_team*.csv')

for csv in csvs:

        match = re.search(r'team_(.*?)_cleaned', csv)
        keyword = match.group(1)

        team_data = pd.read_csv('staging_data/combined_team_stats_cleaned.csv')
        club_data = pd.read_csv('curated_data/club_data_dim.csv')
        

        # Building the Player Sub Dimension Tables as part of the snowflake schema

        # Sub Dim
        team_sub_data = pd.read_csv(f'staging_data/combined_team_{keyword}_cleaned.csv')
        
        if keyword == 'placing':
                team_sub_data['squad'] = team_sub_data['squad'].str.strip()

        if 'uefa_rank' not in keyword:

                query = """
                
                        SELECT  a.club_id,
                                a.club,
                                b.*
                        FROM club_data a
                        LEFT JOIN team_sub_data b
                        on a.club=b.squad and a.league=b.league and a.season=b.season

                        """

        else:
                query = """
                
                        SELECT  a.club_id,
                                a.club,
                                b.*
                        FROM club_data a
                        LEFT JOIN team_sub_data b
                        on a.club=b.squad and a.season=b.season
                        ORDER BY club, season

                        """

        df = sqldf(query, locals())

        df = df.drop(columns = ['squad']).rename(columns={'#_of_players': 'num_of_players'})

        if keyword == 'defense':
                keyword = 'defence'
        elif keyword == 'uefa_rank':
                df['uefa_rating'] = df['uefa_rating'].fillna(-1).astype('int').astype('Int64')
                df['overall_uefa_rating'] = df['overall_uefa_rating'].fillna(-1).astype('int').astype('Int64')
                df['uefa_season_ranking'] = df['uefa_season_ranking'].fillna(-1).astype('int').astype('Int64')
                df['overall_uefa_club_rank'] = df['overall_uefa_club_rank'].fillna(-1).astype('int').astype('Int64')

                df = df[['club_id','club','confederation','country','uefa_rating','uefa_season_ranking','overall_uefa_rating','overall_uefa_club_rank','season']]
                
        else:
                pass

        if keyword == 'stats':
                df['possession'] = pd.to_numeric(df['possession'], errors='coerce').astype('Int64')
                df.rename(columns={'possession':'avg_possession'})
                df.to_csv(f'curated_data/club_overall_{keyword}_data_subdim.csv', index=False)
        else:
                df.to_csv(f'curated_data/club_{keyword}_data_subdim.csv', index=False)

        print(f'{keyword} table written to output location')


defence table written to output location
stats table written to output location
placing table written to output location
passing table written to output location
misc table written to output location
uefa_rank table written to output location
shooting table written to output location
keeper table written to output location


In [223]:
dom_comp_data = pd.read_csv('staging_data/combined_dom_comp_cleaned.csv')
intl_comp_data = pd.read_csv('staging_data/combined_intl_comp_cleaned.csv')
team_data = pd.read_csv('staging_data/combined_team_stats_cleaned.csv')
nation_data = pd.read_csv('curated_data/nation_data_dim.csv')


# Need to add more leagues to this list is scraping more leagues
query = """
        SELECT DISTINCT 
                league,
                CASE WHEN league = 'Austrian_Bundesliga' then 'AUT'
                     WHEN league = 'Belgian_Pro_League' then 'BEL'
                     WHEN league = 'Bulgarian_First_League' then 'BUL'
                     WHEN league = 'Bundesliga' then 'GER'
                     WHEN league = 'Danish_Superliga' then 'DEN'
                     WHEN league = 'Ekstraklasa' then 'POL'
                     WHEN league = 'Eredivisie' then 'NED'
                     WHEN league = 'La_Liga' then 'ESP'
                     WHEN league = 'Ligue_1' then 'FRA'
                     WHEN league = 'Premier_League' then 'ENG'
                     WHEN league = 'Primeira_Liga' then 'POR'
                     WHEN league = 'Serbian_SuperLiga' then 'SRB'
                     WHEN league = 'Serie_A' then 'ITA'
                     WHEN league = 'Super_League_Greece' then 'GRE'
                     WHEN league = 'Super_Lig' then 'TUR'
                END as nation_code

        FROM team_data
        ORDER BY league asc
        """

result_df = sqldf(query, locals())
result_df['league_nation_id'] = range(1, len(result_df) +1)

query = """
        SELECT DISTINCT a.*, b.confederation
        FROM result_df a
        LEFT JOIN nation_data b
                on a.nation_code=b.nation_code
        """

result_df2 = sqldf(query, locals())  
result_df2.to_csv(f'staging_data/league_id.csv', index=False)

# Sorts the comps alphabetically

comp_id_query = """

                SELECT DISTINCT competition_name
                FROM dom_comp_data
                ORDER BY competition_name
                """
comp_id = sqldf(comp_id_query, locals())
comp_id['comp_id'] = range(1, len(comp_id) +1)

intl_comp_id_query = """

                SELECT DISTINCT competition_name
                FROM intl_comp_data
                ORDER BY competition_name
                """
intl_comp_id = sqldf(intl_comp_id_query, locals())
intl_comp_id['intl_comp_id'] = range(1, len(intl_comp_id) +1)

dom_comp_query = """
                SELECT  *
                FROM
                ( SELECT DISTINCT 
                                c.comp_id,
                                a.competition_name as competition,
                                a.country as nation,
                                b.league_nation_id,
                                b.confederation,
                                a.champion,
                                a.top_scorer,
                                a.season

                        FROM dom_comp_data a
                        LEFT JOIN result_df2 b
                                on a.country=b.nation_code
                        LEFT JOIN comp_id c
                                on a.competition_name=c.competition_name
                        order by a.competition_name asc
                )
                WHERE league_nation_id <> " "
                """

intl_comp_query = """
                         SELECT  
                                b.intl_comp_id,
                                a.competition_name as competition,
                                a.governing_body as confederation,
                                a.champion,
                                a.top_scorer,
                                a.season
   
                        FROM intl_comp_data a
                        LEFT JOIN intl_comp_id b
                                on a.competition_name=b.competition_name
                        ORDER BY a.competition_name asc             
                        """
dom_comp_df = sqldf(dom_comp_query, locals())
intl_comp_df = sqldf(intl_comp_query, locals())



# Re-order the columns
dom_comp_df = dom_comp_df[['comp_id','competition','league_nation_id','nation','confederation','champion','top_scorer','season']]

dom_comp_df.to_csv(f'curated_data/dom_comp_data_dim.csv', index=False)
intl_comp_df.to_csv(f'curated_data/intl_comp_data_dim.csv', index=False)

In [59]:
# Creating the fact table 
player_data = pd.read_csv('curated_data/player_data_dim.csv')
club_data = pd.read_csv('curated_data/club_data_dim.csv')
nation_data = pd.read_csv('curated_data/nation_data_dim.csv')
dom_comp_data = pd.read_csv('curated_data/dom_comp_data_dim.csv')
intl_comp_data = pd.read_csv('curated_data/intl_comp_data_dim.csv')




query = """
        SELECT  a.player_id,
                c.club_id, 
                d.nation_id, 
                e.league_nation_id,
                f.comp_id,
                a.position, 
                a.age, 
                a.year_of_birth

        FROM player_data a

        LEFT JOIN club_id_data b
                on a.club = b.club
        LEFT JOIN nation_id_data c
                on a.nation = c.nation_code
        LEFT JOIN dom_comp_data d
                on a.nation = d.nation
        LEFT JOIN intl_comp_data e
                on d.confederation = e.confederation

        ORDER BY a.season, a.squad, a.player
        """

result_df = sqldf(query, locals())

# Handling missing values by assigning them a default -1, allowing for the columns to be the correct type
result_df['nation_id'] = np.nan_to_num(result_df['nation_id'], nan=-1, posinf=-1, neginf=0).astype(int)
result_df['age'] = np.nan_to_num(result_df['age'], nan=-1, posinf=-1, neginf=-1).astype(int)
result_df['year_of_birth'] = np.nan_to_num(result_df['year_of_birth'], nan=-1, posinf=-1, neginf=-1).astype(int)

# Creating PK column
result_df['id'] = result_df.index

# Re-order the columns
result_df = result_df[['id','player_id','club_id','nation_id','league_nation_id','comp_id','position','age','year_of_birth']]

result_df.to_csv(f'curated_data/football_data_fact.csv', index=False)

                

In [ ]:

# Creating